# Imports

In [24]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16

from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

# Configuraciones iniciales de algunas constantes

In [25]:
CATEGORIAS = 'buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'

TRAIN_DIR = Path('train')
TEST_DIR = Path('test')
SIZE = 150

### Análisis de la calidad de las imágenes que serán utilizadas para entrenar

Antes de comenzar a trabajar, revisamos en general las imagenes de train y nos encontramos con muchas imagenes que nada tenían que ver con la categoría a la que decian pertenecer, principalmente en **glacier**. Las mismas fueron removidas de dicho directorio y las guardamos en la carpeta deletes.

Un ejemplo para que se comprenda el tipo de imágenes que quitamos, es el de una persona disfrazada de dinosaurio en el gran cañón categorizada como **glacier**.

## Dividimos las imagenes de train en dos dataset de train y validation

Desarrollamos una función que extrae el 20% de las imagenes de cada categoría para que pasen a ser imagenes de validation. Utilizamos el porcentaje y no una cantidad fija para mantener las proporciones.

In [26]:
# import shutil

# def dividir_train_validation_existente(carpeta_train, carpeta_validation, porcentaje_validacion=0.2):

#     if not os.path.exists(carpeta_validation):
#         os.makedirs(carpeta_validation)

#     for categoria in os.listdir(carpeta_train):
#         ruta_categoria_train = os.path.join(carpeta_train, categoria)
    
#         if os.path.isdir(ruta_categoria_train):
#             imagenes = [img for img in os.listdir(ruta_categoria_train) if os.path.isfile(os.path.join(ruta_categoria_train, img))]

#             num_validacion = int(len(imagenes) * porcentaje_validacion)
        
#             ruta_categoria_validation = os.path.join(carpeta_validation, categoria)
#             if not os.path.exists(ruta_categoria_validation):
#                 os.makedirs(ruta_categoria_validation)

#             for i in range(num_validacion):
#                 imagen = imagenes[i]
#                 ruta_imagen = os.path.join(ruta_categoria_train, imagen)
#                 shutil.move(ruta_imagen, os.path.join(ruta_categoria_validation, imagen))

#     print("División completada: 20% de las imágenes movidas a validation.")

# carpeta_train = 'train'
# carpeta_validation = 'validation'

# dividir_train_validation_existente(carpeta_train, carpeta_validation)


# Análisis exploratorio del conjunto de datos

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

def contar_imagenes_por_categoria(carpeta_train):
    conteo_categorias = {}

    for categoria in os.listdir(carpeta_train):
        ruta_categoria = os.path.join(carpeta_train, categoria)

        if os.path.isdir(ruta_categoria):
            cantidad_imagenes = len([img for img in os.listdir(ruta_categoria) if os.path.isfile(os.path.join(ruta_categoria, img))])
            conteo_categorias[categoria] = cantidad_imagenes

    return conteo_categorias

def graficar_distribucion(conteo_categorias):
    categorias = list(conteo_categorias.keys())
    cantidades = list(conteo_categorias.values())

    plt.figure(figsize=(10, 6))
    sns.barplot(x=categorias, y=cantidades, palette='viridis')


    plt.title('Distribución de Imágenes por Categoría en la Carpeta Test', fontsize=14)
    plt.xlabel('Categoría', fontsize=12)
    plt.ylabel('Cantidad de Imágenes', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

conteo_categorias = contar_imagenes_por_categoria(TRAIN_DIR)

graficar_distribucion(conteo_categorias)

1. *Volumetría de los datos y distribución de las variables a predecir:*
   El conjunto de datos cuenta con un total aproximado de 14,000 imágenes, distribuidas en 6 categorías, de la siguiente manera:

   - *Buildings*: 15.6%
   - *Forest*: 16.2%
   - *Glacier*: 17.1%
   - *Mountain*: 17.9%
   - *Sea*: 16.2%
   - *Street*: 17.0%

   Las categorías están relativamente balanceadas, con diferencias menores en el número de imágenes entre clases. Ninguna categoría domina el conjunto de datos, lo que favorece el entrenamiento de un modelo equilibrado. Esto implica que no sería necesario realizar ajustes significativos para balancear las clases en este punto.

2. *Estructura y tipo de las imágenes:*
   - Las imágenes tienen un tamaño de *150x150 píxeles*.
   - El formato de las imágenes es JPG.
   - Las imágenes pertenecen a paisajes y escenas específicas de categorías bien diferenciadas, como edificios, naturaleza, y calles.


**Aclaración:** la distrución, luego de dividir train en dos sets nuevos, sigue siendo la misma proporción dado que nos quedamos con el 20% de cada categoría en validation y el 80% de cada categoría en train.

#### Para comenzar, trabajamos con imágenes reescaladas, con el brillo cambiado y rotadas

In [28]:
# images_reader = ImageDataGenerator(
#     rescale=1/255,
#     rotation_range=10,
#     brightness_range=(0.5, 1.5),
#     # width_shift_range=0.3,
#     # height_shift_range=0.3,
#     # horizontal_flip=True,
#     # fill_mode='nearest'    
# )

images_reader = ImageDataGenerator(
    rescale=1/255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.5, 1.5),
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    #vertical_flip=True,
    fill_mode='nearest'
)
  
READ_PARAMS = dict(
    class_mode="categorical",
    classes=CATEGORIAS,
    target_size=(SIZE, SIZE),
    color_mode="rgb",
)

In [29]:
VALIDATION_DIR='validation'

Inicializamos los generadores para utilizar las imágenes durante el entrenamiento

In [ ]:
train = images_reader.flow_from_directory(TRAIN_DIR, **READ_PARAMS)
validation = images_reader.flow_from_directory(VALIDATION_DIR, **READ_PARAMS)

#### Ejemplos de imagenes a utilizar

In [31]:
def sample_images(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = next(dataset)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(CATEGORIAS[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
sample_images(train)

In [ ]:
sample_images(validation)

# Modelado

## Entrenamiento de los modelos

In [34]:
input_shape = (SIZE, SIZE, 3)

In [67]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()

#### MLP - Multi layer perceptron

Definimos la **arquitectura** del modelo

In [ ]:
modelMLP = Sequential([
    Input(input_shape),
    
    Flatten(),

    Dense(500, activation='tanh'),
    Dropout(0.25),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelMLP.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelMLP.summary()

Entrenamos el MLP con 5 epocas

In [ ]:
historyMLP = modelMLP.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Graficamos la salida de la corrida

In [ ]:
plt.plot(historyMLP.history['accuracy'], label='train')
plt.plot(historyMLP.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

#### Convolucional

Modelo convolucional con 

In [ ]:
model_weights_at_epochs = {}

modelConvolucional = Sequential([
    Input(input_shape),

    Convolution2D(filters=10, kernel_size=(4, 4), strides=1, activation='relu'),
    Dropout(0.25),
    
    Convolution2D(filters=10, kernel_size=(4, 4), strides=1, activation='relu'),
    Dropout(0.5),
    
    MaxPooling2D(pool_size=(4, 4)),
    
    Flatten(),
    
    Dense(100, activation='tanh'),
    Dropout(0.25),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelConvolucional.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelConvolucional.summary()

In [ ]:
historyConvolutional = modelConvolucional.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

In [ ]:
plt.plot(historyConvolutional.history['accuracy'], label='train')
plt.plot(historyConvolutional.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

### Convolucional modificado

Este modelo convolucional utiliza más filtros (32, 64 y 128) para capturar características detalladas en diferentes niveles de abstracción, mientras que los kernels más pequeños de 3x3 permiten una mejor eficiencia en la detección de patrones locales. Las capas de MaxPooling con un tamaño de 2x2 reducen el tamaño de las características progresivamente, ayudando a prevenir el sobreajuste. La activación relu en las capas densas mejora la eficiencia en la propagación de gradientes, y el Dropout más alto (0.5) en las últimas capas evita el sobreajuste, lo que hace al modelo más robusto.

In [ ]:
modelConvolucionalV2 = Sequential([
    Input(shape=(150, 150, 3)),

    Convolution2D(filters=32, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Convolution2D(filters=64, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Convolution2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

In [ ]:
modelConvolucionalV2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelConvolucionalV2.summary()

In [ ]:
historyConvolutionalV2 = modelConvolucionalV2.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

In [ ]:
plt.plot(historyConvolutionalV2.history['accuracy'], label='train')
plt.plot(historyConvolutionalV2.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

#### VGG16

In [ ]:
# Convolucional usando convoluciones ya entrenadas de VGG16
pretrained_model = VGG16(input_shape=input_shape, include_top=False)
pretrained_model.trainable = False

modelVGG16 = Sequential([
    pretrained_model,

    Flatten(),

    Dense(100, activation='tanh'),
    Dense(100, activation='tanh'),
    
    Dense(len(CATEGORIAS), activation='softmax'),
])

Xception

In [68]:
from tensorflow.keras.applications import Xception


# base_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))
# base_model.trainable = True

# for layer in base_model.layers[:100]:
#     layer.trainable = False
# for layer in base_model.layers[100:]:
#     layer.trainable = True

# modelVGG16 = Sequential([
#     base_model,

#     Flatten(),

#     Dense(100, activation='tanh'),
    
#     Dense(len(CATEGORIAS), activation='softmax'),
# ])
# FUNCIONA - ver mas epocas
# modelVGG16 = Sequential([
#     base_model,

#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5), 
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(100, activation='relu'),
#     Dense(len(CATEGORIAS), activation='softmax')

# ])
# FUNCIONA - ver mas epocas hasta 10
from tensorflow.keras.layers import LeakyReLU

modelVGG16 = Sequential([
    base_model,

    Flatten(),
    Dense(256),
    LeakyReLU(alpha=0.1),  
    Dropout(0.25),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.25),
    Dense(100),
    LeakyReLU(alpha=0.1),
    Dense(len(CATEGORIAS), activation='softmax')
])

# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, LeakyReLU, Flatten
# from tensorflow.keras.optimizers import SGD, Adam

# # Crear el modelo base pre-entrenado
# base_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# # Congelar todas las capas de Xception inicialmente
# for layer in base_model.layers:
#     layer.trainable = False

# # Construir el modelo completo
# modelVGG16 = Sequential([
#     base_model,
#     GlobalAveragePooling2D(),
#     Dense(256),
#     LeakyReLU(alpha=0.1),
#     Dropout(0.25),
#     Dense(128),
#     LeakyReLU(alpha=0.1),
#     Dropout(0.25),
#     Dense(100),
#     LeakyReLU(alpha=0.1),
#     Dense(len(CATEGORIAS), activation='softmax')
# ])

# # Compilar el modelo con Adam
# modelVGG16.compile(
#     optimizer=Adam(),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# # Entrenar solo las capas superiores
# historyVGG = modelVGG16.fit(
#     train,
#     epochs=5,
#     batch_size=128,
#     validation_data=validation,
#     callbacks=[OurCustomCallback()]
# )

# # Descongelar y hacer fine-tuning en las últimas capas de Xception
# # Imprimir los nombres de capas y sus índices para decidir el ajuste fino
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

# # Descongelar las últimas N capas (por ejemplo, 36 capas superiores)
# for layer in base_model.layers[-36:]:
#     layer.trainable = True

# # Compilar de nuevo con un optimizador de menor tasa de aprendizaje
# modelVGG16.compile(
#     optimizer=SGD(learning_rate=0.0001, momentum=0.9),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# # Reentrenar el modelo con ajuste fino en las capas superiores de Xception
# historyVGG_fine = modelVGG16.fit(
#     train,
#     epochs=10,  # Puedes ajustar este número según lo necesario
#     batch_size=128,
#     validation_data=validation,
#     callbacks=[OurCustomCallback()]
# )


In [ ]:
modelVGG16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelVGG16.summary()

In [70]:
model_weights_at_epochs = {}

In [ ]:
historyVGG = modelVGG16.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

In [ ]:
plt.plot(historyVGG.history['accuracy'], label='train')
plt.plot(historyVGG.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.xlim(-1, 12)
# plt.ylim(0.5, 1.0)
plt.legend(loc='upper left')
plt.show()

Dado lo visto en los resultados del modelo VGG16 obtuvimos que:

- Epoch 1/5
  356s - accuracy: 0.6932 - loss: 0.7998 - val_accuracy: 0.8081 - val_loss: 0.5265
- Epoch 2/5
  387s - accuracy: 0.8632 - loss: 0.3760 - val_accuracy: 0.8487 - val_loss: 0.4195
- Epoch 3/5
  380s - accuracy: 0.8873 - loss: 0.3106 - val_accuracy: 0.7666 - val_loss: 0.6069
- Epoch 4/5
  360s - accuracy: 0.8931 - loss: 0.2758 - val_accuracy: 0.8400 - val_loss: 0.4288
- Epoch 5/5
  359s - accuracy: 0.8969 - loss: 0.2680 - val_accuracy: 0.8342 - val_loss: 0.4899

Elegimos la epoca 2 dado que en las siguientes el modelo overfitea porque el accuracy de validation empieza a caer. 

Prueba LeakyRelu

In [ ]:
from tensorflow.keras.layers import LeakyReLU

pretrained_model = VGG16(input_shape=input_shape, include_top=False)
pretrained_model.trainable = False

model_leaky_relu = Sequential([
    pretrained_model,
    Flatten(),
    Dense(128),
    LeakyReLU (alpha=0.1),
    Dense(len(CATEGORIAS), activation='softmax'),
])

model_leaky_relu.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
model_leaky_relu.summary()

model_weights_at_epochs = {}

historyLeakyRelu = model_leaky_relu.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

In [ ]:
plt.plot(historyLeakyRelu.history['accuracy'], label='train')
plt.plot(historyLeakyRelu.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.xlim(-1, 4)
# plt.ylim(0.5, 1.0)
plt.legend(loc='upper left')
plt.show()

Por ahora el LeakyRelu es el mejor resultado obtenido, donde en la epoca 5 se puede obtener un accuracy del 0.91. 

Tambien se probó cambiando los parametros del images_reader para ver si se obtenian mejores resultados pero no fue el caso, sino que se obtuvo un 0.82 en la epoca 5.

images_reader = ImageDataGenerator(
    
    rescale=1/255,
    rotation_range=10,
    brightness_range=(0.5, 1.5),
    width_shift_range=0.3, *
    height_shift_range=0.3, *
    horizontal_flip=True, *
    fill_mode='nearest' *
) 

*Son los parametros que se agregaron en comparacion a la primer prueba del LeakyRelu. 


In [ ]:
modelConvolucional.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)
modelConvolucional.summary()

In [ ]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()

In [ ]:
historyCon = modelConvolucional.fit(
    train,
    epochs=5,
    batch_size=128,
    validation_data=validation,
    callbacks=[OurCustomCallback()]
)

Vemos el accuracy de ambos conjuntos, tanto train como validation, durante todo el proceso

In [ ]:
plt.plot(historyCon.history['accuracy'], label='train')
plt.plot(historyCon.history['val_accuracy'], label='validation')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

Elegimos la que consideremos como la mejor epoca y nos quedamos con ese conjunto de pesos

In [79]:
BEST_EPOCH = 4
# modelConvolucional.set_weights(model_weights_at_epochs[BEST_EPOCH])
# modelMLP.set_weights(model_weights_at_epochs[BEST_EPOCH])
# model_leaky_relu.set_weights(model_weights_at_epochs[BEST_EPOCH])
modelVGG16.set_weights(model_weights_at_epochs[BEST_EPOCH])


Ahora analizamos el error de ambos conjuntos para sacar nuestras propias conclusiones

In [ ]:
datasets = (
    ('train', images_reader.flow_from_directory(TRAIN_DIR, **READ_PARAMS, batch_size=-1)),
    ('validation', images_reader.flow_from_directory(VALIDATION_DIR, **READ_PARAMS, batch_size=-1)),
)

In [ ]:


for dataset_name, dataset in datasets:
    print('#' * 25, dataset_name, '#' * 25)

    batch_images, batch_labels = next(dataset)
    
    # super importante: usamos argmax para convertir cosas de este formato:
    # [(0, 1, 0), (1, 0, 0), (1, 0, 0), (0, 0, 1)]
    # a este formato (donde tenemos el índice de la clase que tiene número más alto):
    # [1, 0, 0, 2]
    predictions = np.argmax(modelMLP.predict(batch_images), axis=-1)
    labels = np.argmax(batch_labels, axis=-1)
    
    print('Accuracy:', accuracy_score(labels, predictions))

    # graficamos la confussion matrix
    plt.figure(figsize=(3, 4))
        
    plt.xticks([0, 1, 2, 3, 4, 5], CATEGORIAS, rotation=45)
    plt.yticks([0, 1, 2, 3, 4, 5], CATEGORIAS)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.imshow(
        confusion_matrix(labels, predictions), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

In [ ]:
for dataset_name, dataset in datasets:
    print('#' * 25, dataset_name, '#' * 25)

    batch_images, batch_labels = next(dataset)
    
    # super importante: usamos argmax para convertir cosas de este formato:
    # [(0, 1, 0), (1, 0, 0), (1, 0, 0), (0, 0, 1)]
    # a este formato (donde tenemos el índice de la clase que tiene número más alto):
    # [1, 0, 0, 2]
    predictions = np.argmax(modelConvolucional.predict(batch_images), axis=-1)
    labels = np.argmax(batch_labels, axis=-1)
    
    print('Accuracy:', accuracy_score(labels, predictions))

    # graficamos la confussion matrix
    plt.figure(figsize=(3, 4))
        
    plt.xticks([0, 1, 2, 3, 4, 5], CATEGORIAS, rotation=45)
    plt.yticks([0, 1, 2, 3, 4, 5], CATEGORIAS)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.imshow(
        confusion_matrix(labels, predictions), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

In [ ]:
for dataset_name, dataset in datasets:
    print('#' * 25, dataset_name, '#' * 25)

    batch_images, batch_labels = next(dataset)
    
    # super importante: usamos argmax para convertir cosas de este formato:
    # [(0, 1, 0), (1, 0, 0), (1, 0, 0), (0, 0, 1)]
    # a este formato (donde tenemos el índice de la clase que tiene número más alto):
    # [1, 0, 0, 2]
    predictions = np.argmax(modelVGG16.predict(batch_images), axis=-1)
    labels = np.argmax(batch_labels, axis=-1)
    
    print('Accuracy:', accuracy_score(labels, predictions))

    # graficamos la confussion matrix
    plt.figure(figsize=(3, 4))
        
    plt.xticks([0, 1, 2, 3, 4, 5], CATEGORIAS, rotation=45)
    plt.yticks([0, 1, 2, 3, 4, 5], CATEGORIAS)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.imshow(
        confusion_matrix(labels, predictions), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

In [ ]:
for dataset_name, dataset in datasets:
    print('#' * 25, dataset_name, '#' * 25)

    batch_images, batch_labels = next(dataset)
    
    # super importante: usamos argmax para convertir cosas de este formato:
    # [(0, 1, 0), (1, 0, 0), (1, 0, 0), (0, 0, 1)]
    # a este formato (donde tenemos el índice de la clase que tiene número más alto):
    # [1, 0, 0, 2]
    predictions = np.argmax(model_leaky_relu.predict(batch_images), axis=-1)
    labels = np.argmax(batch_labels, axis=-1)
    
    print('Accuracy:', accuracy_score(labels, predictions))

    # graficamos la confussion matrix
    plt.figure(figsize=(3, 4))
        
    plt.xticks([0, 1, 2, 3, 4, 5], CATEGORIAS, rotation=45)
    plt.yticks([0, 1, 2, 3, 4, 5], CATEGORIAS)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.imshow(
        confusion_matrix(labels, predictions), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

Se observa en los gráficos de confusión que el modelo presenta problemas a la hora de tener que clasificar montañas, prediciendo que son oceanos o glaciares. Por ejemplo, al probar el modelo con la imagen 20058.jpg se muestra la prediccion erronea que planteamos.    

## Ahora probaremos con nuestras propias imágenes!

In [ ]:
from IPython.display import Image, display


def show_and_predict(image_path):
    image_array = img_to_array(load_img(image_path, target_size=(SIZE, SIZE)))
    inputs = np.array([image_array])  # armamos un "dataset" con solo esa imagen
    predictions = model_leaky_relu.predict(inputs)
    display(Image(image_path, width=500))
    print("Prediction:", CATEGORIAS[np.argmax(predictions)])
    print("Prediction detail:", predictions)
show_and_predict("./test/20070.jpg")

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

CATEGORIA2 = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
SIZE = 150

def predict_images_from_directory(directory):
    results = [] 
    images = [] 

    for image_name in os.listdir(directory):
        image_path = os.path.join(directory, image_name)

        if image_name.endswith(".jpg"):
            image_array = img_to_array(load_img(image_path, target_size=(SIZE, SIZE)))
            images.append(image_array)

    inputs = np.array(images) / 255.0  

    predictions = modelVGG16.predict(inputs)

    for i, image_name in enumerate(os.listdir(directory)):
        if image_name.endswith(".jpg"):
            predicted_class = CATEGORIA2[np.argmax(predictions[i])]
            results.append([image_name, predicted_class])
    
    df = pd.DataFrame(results, columns=["ID", "Label"])
    df.to_csv("prediccionesXceptionModificaDos.csv", index=False)
    print("Predicciones guardadas en 'prediccionesXceptionModificaDos.csv'")

predict_images_from_directory('test')


# Conclusiones